# Clasificación de textos utilizando algoritmos de Regresión Logística y Random Forest (sklearn)

- El presente tutorial se inspira de: https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

En este tutorial intentaremos clasificar reseñas de productos Amazon (Alexa) en dos categorías: positivos o negativos ("Análisis de sentimientos").

Utilizaremos un enfoque simple:
- representaremos los textos con representaciones vectoriales "Bag of Words"
- utilizaremos algoritmos de Machine Learning para aprender modelos a partir de las representaciones vectoriales
- evaluaremos los modelos utilizando una matriz de confusión

In [1]:
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.7 MB/s eta 0:00:00m eta 0:00:010:0101
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
#NLP
import spacy
nlp = spacy.load("en_core_web_sm")
print(spacy.__version__)
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

#SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression # Regresion Logística

#PANDAS
import pandas as pd

3.6.1


# 2. Dataset: Alexa

Vamos a usar un conjunto de datos real: un conjunto de reseñas de productos de Amazon Alexa. Este conjunto de datos viene como un archivo separado por tabulaciones (.tsv). Tiene cinco columnas: 
- __rating__: se refiere a la calificación que cada usuario dio a Alexa (de 0 a 5). 
- __fecha__: fecha de la reseña
- __variación__: describe el modelo de producto Alexa que el usuario comentó.
- __verified_reviews__: contiene el texto del comentario.
- __feedback__: contiene un label, 0 o 1, que indica el sentimiento general negativo (0) o positivo (1).

In [30]:
# Loading TSV file
df_amazon = pd.read_csv("datasets/amazon_alexa.tsv", sep="\t")

In [31]:
# Top 5 records
df_amazon

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3136,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3137,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3138,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3139,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [35]:
# shape of dataframe
df_amazon.shape

(3140, 5)

In [36]:
# Feedback Value count
df_amazon.feedback.value_counts()

feedback
1    2887
0     253
Name: count, dtype: int64

In [46]:
# Eliminar filas con valores NaN o en blanco en la columna 'verified_reviews'
df_amazon.dropna(subset=['verified_reviews'], inplace=True)

# Convertir una columna a tipo cadena
df_amazon['verified_reviews'] = df_amazon['verified_reviews'].astype(str)


#### Partición de los datos en conjuntos de entrenamiento y test para entrenar y evaluar un modelo predictivo

Usaremos la mitad de nuestro conjunto de datos como nuestro conjunto de entrenamiento, que incluirá las respuestas correctas. Luego probaremos nuestro modelo usando la otra mitad del conjunto de datos sin darle las respuestas, para ver con qué precisión funciona.

Convenientemente, scikit-learn nos da una función incorporada para hacer esto: train_test_split(). Sólo necesitamos decirle el conjunto de características que queremos que se divida (X), las etiquetas contra las que queremos que se realice la prueba (ylabels), y el tamaño que queremos usar para el conjunto de pruebas (representado como un porcentaje en forma decimal).

In [47]:
X = df_amazon['verified_reviews'] # the features we want to analyze
ylabels = df_amazon['feedback'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.5)

In [48]:
X

0                                           Love my Echo!
1                                               Loved it!
2       Sometimes while playing a game, you can answer...
3       I have had a lot of fun with this thing. My 4 ...
4                                                   Music
                              ...                        
3136    Perfect for kids, adults and everyone in betwe...
3137    Listening to music, searching locations, check...
3138    I do love these things, i have them running my...
3139    Only complaint I have is that the sound qualit...
3140                                                 Good
Name: verified_reviews, Length: 3140, dtype: object

# 3. Preprocesamientos y representación vectorial

Crearemos una función personalizada <code>spacy_tokenizer()</code> que acepta una frase como entrada y la procesa en tokens, realizando lemmatización, minúsculas y eliminando palabras stop-words.

In [49]:
# Create our list of punctuation marks
punctuations = [".",",","!","?", "#","&"]

# Create our list of stopwords
stop_words=[""]

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lower_ for word in mytokens]
        
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

#### Vectorización de los textos en BoW o TF-IDF, con scikit-learn

Podemos generar una matriz BoW para nuestros datos de texto usando la clase <code>CountVectorizer</code> de scikit-learn. En el código de abajo, le decimos a CountVectorizer que use la función personalizada spacy_tokenizer que construimos como su tokenizer, y que defina el rango de ngramo que queremos.

Los N-gramos son combinaciones de palabras adyacentes en un texto dado, donde _n_ es el número de palabras que se incluyen en las fichas. Por ejemplo, en la frase "¿Quién ganará la Copa del Mundo de fútbol en 2022? Bigramas sería una secuencia de dos palabras contiguas como "quién ganará", "ganará la", y así sucesivamente. Así que el parámetro ngram_range que usaremos en el código de abajo establece los límites inferior y superior de nuestros ngramas (usaremos unigramas). Entonces asignaremos los ngramas a bow_vector.

In [50]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

Podemos también transformar los textos en vectores para tener los pesos TF-IDF de cada palabra en cada documento:

In [51]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

# 4. Entrenamiento del modelo de clasificación

Es el momento de construir nuestro modelo predictivo. Empezaremos importando el módulo LogisticRegression y creando un objeto clasificador LogisticRegression.

Luego, crearemos un pipeline de procesamiento con dos componentes: un vectorizador y algoritmo de clasificación basado en la regresión logística. El vectorizador utiliza preprocesamientos (spacy) y vectorización (scikit-learn) para crear una matriz para representar nuestros textos.

Una vez que se construya este pipeline, se aprende el modelo predictivo llamando el método fit().

- Regresión Logística sobre representacion vectorial "BoW"

In [52]:
# Logistic Regression Classifier
modelLR = LogisticRegression()

# Create pipeline using Bag of Words
model1 = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelLR)])

# model generation
model1.fit(X_train,y_train)

/home/matthieu/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7fc9d20a6290>)),
                ('regression-ML', LogisticRegression())])

- Regresión Logística sobre representacion vectorial "TF-IDF"

In [53]:
# Logistic Regression Classifier
modelLR = LogisticRegression()

model2 = Pipeline([('preprocessing', tfidf_vector),
                 ('regression-ML', modelLR)])

# model generation
model2.fit(X_train,y_train)

/home/matthieu/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocessing',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7fc9d20a6290>)),
                ('regression-ML', LogisticRegression())])

- Random Forest sobre representacion vectorial "BoW"

In [54]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=0)

model3 = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelRF)])

# model generation
model3.fit(X_train,y_train)

/home/matthieu/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7fc9d20a6290>)),
                ('regression-ML', RandomForestClassifier(random_state=0))])

# 5. Evaluación del modelo de clasificación

In [55]:
# Predicting with a test dataset
predicted = model1.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

[1 1 0 ... 1 0 1]
Logistic Regression Accuracy: 0.9375796178343949
Logistic Regression Precision: 0.9495016611295681
Logistic Regression Recall: 0.9848380427291523


In [56]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)

#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

[[  43   76]
 [  22 1429]]
              precision    recall  f1-score   support

           0       0.66      0.36      0.47       119
           1       0.95      0.98      0.97      1451

    accuracy                           0.94      1570
   macro avg       0.81      0.67      0.72      1570
weighted avg       0.93      0.94      0.93      1570



In [59]:
def printNMostInformative(vectorizer, model, N):
    feature_names = vectorizer.get_feature_names_out()  # Usar get_feature_names_out() en lugar de get_feature_names()
    coefs_with_fns = sorted(zip(model.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best:")
    for feat in topClass1:
        print(feat)
    print("Class 2 best:")
    for feat in topClass2:
        print(feat)


In [60]:
printNMostInformative(bow_vector, modelLR, 20)

Class 1 best:
(-2.864253831937096, 'not')
(-1.645463329498088, 'n’t')
(-1.5072789322479718, 'does')
(-1.4700752552416207, 'back')
(-1.4592562290800892, 'cheap')
(-1.3788646435913796, 'poor')
(-1.2224452527417788, 'amazon')
(-1.1556696173598637, 'working')
(-1.1495984521788944, ' ')
(-1.144347381494547, "n't")
(-1.04832681372679, 'after')
(-1.030154936764939, 'returned')
(-1.0195199498113592, 'when')
(-0.960984111269652, 'refurbished')
(-0.9410120956320043, 'need')
(-0.9365833692632937, 'low')
(-0.9206267228015528, 'return')
(-0.8880300228915343, 'no')
(-0.8848122760063369, 'from')
(-0.8517794415936553, 'stopped')
Class 2 best:
(2.814308992489963, 'love')
(2.0491100829506994, 'great')
(1.4207825377638361, 'easy')
(1.0420283881302814, 'we')
(0.9878299893879765, 'my')
(0.9771516086951332, 'works')
(0.86089590544353, 'like')
(0.8442124061363857, 'with')
(0.7877614924964642, 'perfect')
(0.7867334975170295, 'music')
(0.7028039203140346, 'fun')
(0.6548064701580739, 'set')
(0.6347736677911906,